In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys, os, gc, types
import time

In [2]:
import os

root_paths = [
    "/data/kaggle-instacart",
    "/Users/jiayou/Dropbox/珺珺的程序/Kaggle/Instacart",
    "/Users/jiayou/Dropbox/Documents/珺珺的程序/Kaggle/Instacart"
]
root = None
for p in root_paths:
    if os.path.exists(p):
        root = p
        break
path = root
sbpath = os.path.join(root, 'sb')

In [3]:
class tick_tock:
    def __init__(self, process_name, verbose=1):
        self.process_name = process_name
        self.verbose = verbose
    def __enter__(self):
        if self.verbose:
            print(self.process_name + " starts...")
            self.begin_time = time.time()
    def __exit__(self, type, value, traceback):
        if self.verbose:
            end_time = time.time()
            print('{} done: {:.2f}s'.format(self.process_name, end_time - self.begin_time))

In [4]:
import gc
import pandas as pd
import numpy as np
import os
import json
import sklearn.metrics
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from scipy.sparse import dok_matrix, coo_matrix
from sklearn.utils.multiclass import  type_of_target

In [5]:
gorders = pd.read_csv(os.path.join(path, "orders.csv"), dtype={'order_id': np.uint32,
                                                              'user_id': np.uint32,
                                                              'eval_set': 'category',
                                                              'order_number': np.uint8,
                                                              'order_dow': np.uint8,
                                                              'order_hour_of_day': np.uint8
                                                              })

gorder_prior = pd.read_csv(os.path.join(path, "order_products__prior.csv"), dtype={'order_id': np.uint32,
                                                                                  'product_id': np.uint16,
                                                                                  'add_to_cart_order': np.uint8,
                                                                                  'reordered': bool})
gorder_prior = gorder_prior.merge(gorders[['order_id', 'user_id']], on='order_id', how='left')

gorder_train = pd.read_csv(os.path.join(path, "order_products__train.csv"), dtype={'order_id': np.uint32,
                                                                                  'product_id': np.uint16,
                                                                                  'add_to_cart_order': np.uint8,
                                                                                  'reordered': bool})
gorder_train = gorder_train.merge(gorders[['order_id', 'user_id']], on='order_id', how='left')

aisles = pd.read_csv(os.path.join(path, "aisles.csv"), dtype={'aisle_id': np.uint8, 'aisle':'category'})
departments = pd.read_csv(
    os.path.join(path, "departments.csv"), 
    dtype={'department_id':np.uint8, 'department': 'category'})
products = pd.read_csv(os.path.join(path, "products.csv"), dtype={'product_id': np.uint16,
                                                                  'aisle_id': np.uint8,
                                                                  'department_id': np.uint8})

In [6]:
def aug_name(s, ms):
    return 'aug{}-{}'.format(s, ms)

def orig_name(shard):
    return 'orig{}'.format(shard)

def load(shard=None, nshards=None, aug=None):
    if aug is None:
        global gorders, gorder_prior, gorder_train
        orders = gorders.loc[gorders.user_id % nshards == shard, :]
        
        order_prior = gorder_prior.loc[gorder_prior.user_id % nshards == shard, :]
        order_prior.drop('user_id', inplace = True, axis=1)
        
        order_train = gorder_train.loc[gorder_train.user_id % nshards == shard, :]
        order_train.drop('user_id', inplace = True, axis=1)
    else:
        pf = os.path.join(root, 'aug', 'order_products__prior.{}.csv'.format(aug))
        tf = os.path.join(root, 'aug', 'order_products__train.{}.csv'.format(aug))
        of = os.path.join(root, 'aug', 'orders.{}.csv'.format(aug))
        orders = pd.read_csv(of, dtype={
            'order_id': np.uint32,
            'user_id': np.uint32,
            'eval_set': 'category',
            'order_number': np.uint8,
            'order_dow': np.uint8,
            'order_hour_of_day': np.uint8
        })
        order_prior = pd.read_csv(pf, dtype={
            'order_id': np.uint32,
            'product_id': np.uint16,
            'add_to_cart_order': np.uint8,
            'reordered': bool
        })
        order_train = pd.read_csv(tf, dtype={
            'order_id': np.uint32,
            'product_id': np.uint16,
            'add_to_cart_order': np.uint8,
            'reordered': bool
        })
    return (orders, order_prior, order_train)

In [7]:
def process_shard(s):
    aug = s[0]
    if aug:
        orders, order_prior, order_train = load(aug=aug_name(s[1], s[2]))
        sname = aug_name(s[1], s[2])
    else:
        orders, order_prior, order_train = load(shard=s[1], nshards=s[2])
        sname = orig_name(s[1])
        
    global aisles, departments, products

########## user's product list

# def create_products():

#     print('loaded')

    torders = orders.loc[orders.eval_set == 'prior', :]
    orders_user = torders[['order_id', 'user_id']]
    previous_products = pd.merge(order_prior, orders_user, on='order_id')
    previous_products = previous_products.loc[:, ['user_id', 'product_id']].drop_duplicates()

#     print('save')
#     print(labels.shape)
#     print(labels.columns)
#     labels.to_pickle('sb/previous_products.pkl')
#     return labels

# labels = create_products()



########## abt skeleton without features

# def split_data_set():
    labels = previous_products.copy()
    torders = orders.loc[(orders.eval_set == 'train') | (orders.eval_set == 'test'), :]
    labels = pd.merge(labels, torders[['order_id', 'user_id', 'eval_set']], on='user_id').drop(['user_id'], axis=1)

#     print('data is loaded')

    current = np.unique(labels.order_id)
    
    current = labels.loc[np.in1d(labels.order_id, current), :]

    current = pd.merge(
        order_train.drop(['add_to_cart_order'], axis=1), 
        current, on=['order_id', 'product_id'], how='right')
    current.reordered.fillna(False, inplace=True)
    chunk = current

# folds = split_data_set()



########## intervals a user purchases a product

# def orders_comsum():
#     path = root
#     sbpath = os.path.join(root, 'sb')

    labels = chunk.copy()
    user_product = previous_products.copy()

    order_comsum = orders[['user_id', 'order_number', 'days_since_prior_order']].groupby(['user_id', 'order_number'])\
        ['days_since_prior_order'].sum().groupby(level=[0]).cumsum()\
        .reset_index().rename(columns={'days_since_prior_order':'days_since_prior_order_comsum'})

    # order_comsum['days_since_prior_order_comsum'].fillna(0, inplace=True)
#     order_comsum.to_pickle('sb/orders_comsum.pkl')

    order_comsum = pd.merge(
        order_comsum, orders, 
        on=['user_id', 'order_number'])[['user_id', 'order_number', 'days_since_prior_order_comsum', 'order_id']]

    order_product = pd.merge(order_prior, orders, on='order_id')[['order_id', 'product_id', 'eval_set']]
    order_product_train_test = labels[['order_id', 'product_id', 'eval_set']]

    order_product = pd.concat([order_product, order_product_train_test])

    order_product = pd.merge(order_product, order_comsum, on='order_id')

#     print(order_product.columns)

    order_product = pd.merge(order_product, user_product, on=['user_id', 'product_id'])

    temp = order_product.groupby(['user_id', 'product_id', 'order_number'])\
        ['days_since_prior_order_comsum'].sum().groupby(level=[0, 1]).apply(lambda x: np.diff(np.nan_to_num(x)))
    temp = temp.to_frame('periods').reset_index()

#     temp.to_pickle('sb/product_period.pkl')

    aggregated = temp.copy()
    aggregated['last'] = aggregated.periods.apply(lambda x: x[-1])
    aggregated['prev1'] = aggregated.periods.apply(lambda x: x[-2] if len(x) > 1 else np.nan)
    aggregated['prev2'] = aggregated.periods.apply(lambda x: x[-3] if len(x) > 2 else np.nan)
    aggregated['median'] = aggregated.periods.apply(lambda x: np.median(x[:-1]))
    aggregated['mean'] = aggregated.periods.apply(lambda x: np.mean(x[:-1]))
    aggregated.drop('periods', axis=1, inplace=True)

    if aug:
        aggregated.to_pickle('abt/feature.up_interval_stat.{}.pkl'.format(sname))
    else:
        aggregated.to_pickle('abt-share/feature.up_interval_stat.{}.pkl'.format(sname))
    
#     return (order_comsum, temp, aggregated)

# oc = orders_comsum()



########## user_dep_stat and user_aisle_stat

# def user_product_rank():
#     path = root
#     sbpath = os.path.join(root, 'sb')

    order_train = chunk.copy()
    
    orders_products = pd.merge(orders, order_prior, on="order_id")

    orders_products_products = pd.merge(orders_products, products[['product_id', 'department_id', 'aisle_id']],
                                        on='product_id')

    user_dep_stat = orders_products_products.groupby(['user_id', 'department_id']).agg(
        {'product_id': lambda x: x.nunique(),
         'reordered': 'sum'
         })
    user_dep_stat.rename(columns={'product_id': 'dep_products',
                                  'reordered': 'dep_reordered'}, inplace=True)
    user_dep_stat.reset_index(inplace=True)
#     print(user_dep_stat.columns)
#     user_dep_stat.to_pickle('abt-share/feature.user_department.{}.pkl'.format(sname))

    user_aisle_stat = orders_products_products.groupby(['user_id', 'aisle_id']).agg(
        {'product_id': lambda x: x.nunique(),
         'reordered': 'sum'
         })
    user_aisle_stat.rename(columns={'product_id': 'aisle_products',
                                    'reordered': 'aisle_reordered'}, inplace=True)
    user_aisle_stat.reset_index(inplace=True)
#     print(user_aisle_stat.columns)
#     user_aisle_stat.to_pickle('abt-share/feature.user_aisle.{}.pkl'.format(sname))
    
#     return (user_dep_stat, user_aisle_stat)

# upr = user_product_rank()



########## other features

    order_train = chunk.copy()
    order_test = order_train.loc[order_train.eval_set == "test", ['order_id', 'product_id']]
    order_train = order_train.loc[order_train.eval_set == "train", ['order_id',  'product_id',  'reordered']]

    ###########################

#     prob = pd.merge(order_prior, orders, on='order_id')
# #     print(prob.columns)
#     prob = prob.groupby(['product_id', 'user_id'])\
#         .agg({'reordered':'sum', 'user_id': 'size'})
# #     print(prob.columns)

#     prob.rename(columns={'sum': 'reordered',
#                          'user_id': 'total'}, inplace=True)

#     prob.reordered = (prob.reordered > 0).astype(np.float32)
#     prob.total = (prob.total > 0).astype(np.float32)
#     prob['reorder_prob'] = prob.reordered / prob.total
#     prob = prob.groupby('product_id').agg({'reorder_prob': 'mean'}).rename(columns={'mean': 'reorder_prob'})\
#         .reset_index()


#     prod_stat = order_prior.groupby('product_id').agg({'reordered': ['sum', 'size'],
#                                                        'add_to_cart_order':'mean'})
#     prod_stat.columns = prod_stat.columns.levels[1]
#     prod_stat.rename(columns={'sum':'prod_reorders',
#                               'size':'prod_orders',
#                               'mean': 'prod_add_to_card_mean'}, inplace=True)
#     prod_stat.reset_index(inplace=True)

#     prod_stat['reorder_ration'] = prod_stat['prod_reorders'] / prod_stat['prod_orders']

#     prod_stat = pd.merge(prod_stat, prob, on='product_id')

#     # prod_stat.drop(['prod_reorders'], axis=1, inplace=True)

    user_stat = orders.loc[orders.eval_set == 'prior', :].groupby('user_id').agg({'order_number': 'max',
                                                                                  'days_since_prior_order': ['sum',
                                                                                                             'mean',
                                                                                                             'median']})
    user_stat.columns = user_stat.columns.droplevel(0)
    user_stat.rename(columns={'max': 'user_orders',
                              'sum': 'user_order_starts_at',
                              'mean': 'user_mean_days_since_prior',
                              'median': 'user_median_days_since_prior'}, inplace=True)
    user_stat.reset_index(inplace=True)

    orders_products = pd.merge(orders, order_prior, on="order_id")

    user_order_stat = orders_products.groupby('user_id').agg({'user_id': 'size',
                                                              'reordered': 'sum',
                                                              "product_id": lambda x: x.nunique()})

    user_order_stat.rename(columns={'user_id': 'user_total_products',
                                    'product_id': 'user_distinct_products',
                                    'reordered': 'user_reorder_ratio'}, inplace=True)

    user_order_stat.reset_index(inplace=True)
    user_order_stat.user_reorder_ratio = user_order_stat.user_reorder_ratio / user_order_stat.user_total_products

    user_stat = pd.merge(user_stat, user_order_stat, on='user_id')
    user_stat['user_average_basket'] = user_stat.user_total_products / user_stat.user_orders

    ########################### products

#     prod_usr = orders_products.groupby(['product_id']).agg({'user_id': lambda x: x.nunique()})
#     prod_usr.rename(columns={'user_id':'prod_users_unq'}, inplace=True)
#     prod_usr.reset_index(inplace=True)

#     prod_usr_reordered = orders_products.loc[orders_products.reordered, :]\
#         .groupby(['product_id']).agg({'user_id': lambda x: x.nunique()})
#     prod_usr_reordered.rename(columns={'user_id': 'prod_users_unq_reordered'}, inplace=True)
#     prod_usr_reordered.reset_index(inplace=True)

    order_stat = orders_products.groupby('order_id').agg({'order_id': 'size'}) \
        .rename(columns={'order_id': 'order_size'}).reset_index()

    orders_products = pd.merge(orders_products, order_stat, on='order_id')
    orders_products['add_to_cart_order_inverted'] = orders_products.order_size - orders_products.add_to_cart_order
    orders_products['add_to_cart_order_relative'] = orders_products.add_to_cart_order / orders_products.order_size

    data = orders_products.groupby(['user_id', 'product_id']).agg({'user_id': 'size',
                                                                   'order_number': ['min', 'max'],
                                                                   'add_to_cart_order': ['mean', 'median'],
                                                                   'days_since_prior_order': ['mean', 'median'],
                                                                   'order_dow': ['mean', 'median'],
                                                                   'order_hour_of_day': ['mean', 'median'],
                                                                   'add_to_cart_order_inverted': ['mean', 'median'],
                                                                   'add_to_cart_order_relative': ['mean', 'median'],
                                                                   'reordered': ['sum']})

    data.columns = data.columns.droplevel(0)
    data.columns = ['up_orders', 'up_first_order', 'up_last_order', 'up_mean_cart_position', 'up_median_cart_position',
                    'days_since_prior_order_mean', 'days_since_prior_order_median', 'order_dow_mean',
                    'order_dow_median',
                    'order_hour_of_day_mean', 'order_hour_of_day_median',
                    'add_to_cart_order_inverted_mean', 'add_to_cart_order_inverted_median',
                    'add_to_cart_order_relative_mean', 'add_to_cart_order_relative_median',
                    'reordered_sum'
                    ]

    data['user_product_reordered_ratio'] = (data.reordered_sum + 1.0) / data.up_orders

    # data['first_order'] = data['up_orders'] > 0
    # data['second_order'] = data['up_orders'] > 1
    #
    # data.groupby('product_id')['']

    data.reset_index(inplace=True)

#     data = pd.merge(data, prod_stat, on='product_id')
    data = pd.merge(data, user_stat, on='user_id')

    data['up_order_rate'] = data.up_orders / data.user_orders
    data['up_orders_since_last_order'] = data.user_orders - data.up_last_order
    data['up_order_rate_since_first_order'] = data.user_orders / (data.user_orders - data.up_first_order + 1)

    ############################

#     user_dep_stat = pd.read_pickle('data/user_department_products.pkl')
#     user_aisle_stat = pd.read_pickle('data/user_aisle_products.pkl')

    ############### train

#     print(order_train.shape)
    order_train = pd.merge(order_train, products, on='product_id')
#     print(order_train.shape)
    order_train = pd.merge(order_train, orders, on='order_id')
#     print(order_train.shape)
    order_train = pd.merge(order_train, user_dep_stat, on=['user_id', 'department_id'])
#     print(order_train.shape)
    order_train = pd.merge(order_train, user_aisle_stat, on=['user_id', 'aisle_id'])
#     print(order_train.shape)

#     order_train = pd.merge(order_train, prod_usr, on='product_id')
#     print(order_train.shape)
#     order_train = pd.merge(order_train, prod_usr_reordered, on='product_id', how='left')
#     order_train.prod_users_unq_reordered.fillna(0, inplace=True)
#     print(order_train.shape)

    order_train = pd.merge(order_train, data, on=['product_id', 'user_id'])
#     print(order_train.shape)

    order_train['aisle_reordered_ratio'] = order_train.aisle_reordered / order_train.user_orders
    order_train['dep_reordered_ratio'] = order_train.dep_reordered / order_train.user_orders

#     order_train = pd.merge(order_train, product_periods, on=['user_id',  'product_id'])

    if aug:
        order_train.to_pickle(os.path.join(root, 'abt', 'abt_train.sb.{}.pkl'.format(sname)))
    else:
        order_train.to_pickle(os.path.join(root, 'abt-share', 'abt_train.sb.{}.pkl'.format(sname)))

    ##############

    if not aug:
        order_test = pd.merge(order_test, products, on='product_id')
        order_test = pd.merge(order_test, orders, on='order_id')
        order_test = pd.merge(order_test, user_dep_stat, on=['user_id', 'department_id'])
        order_test = pd.merge(order_test, user_aisle_stat, on=['user_id', 'aisle_id'])

#         order_test = pd.merge(order_test, prod_usr, on='product_id')
#         order_test = pd.merge(order_test, prod_usr_reordered, on='product_id', how='left')
#         order_train.prod_users_unq_reordered.fillna(0, inplace=True)

        order_test = pd.merge(order_test, data, on=['product_id', 'user_id'])

        order_test['aisle_reordered_ratio'] = order_test.aisle_reordered / order_test.user_orders
        order_test['dep_reordered_ratio'] = order_test.dep_reordered / order_test.user_orders

#         order_test = pd.merge(order_test, product_periods, on=['user_id', 'product_id'])
    
        order_test.to_pickle(os.path.join(root, 'abt-share', 'abt_test.sb.{}.pkl'.format(sname)))

#     order_train = pd.merge(order_train, product_embeddings, on=['product_id'])
#     order_test = pd.merge(order_test, product_embeddings, on=['product_id'])


    return (aggregated, order_train, order_test)

In [8]:
def process_shards(shards):
    for s in shards:
        with tick_tock("Process shard {}".format(s)):
            process_shard(shard=s)
        gc.collect()

In [ ]:
from multiprocessing import Process

n_processes = 32

shards = [(False, s, 64) for s in range(64)] + [(True, s, ms) for ms in range(2) for s in range(32)]

jobs = []
for pid in range(n_processes):
    cur_shards = [shards[i] for i in range(len(shards)) if i % n_processes == pid]
    p = Process(target=process_shards, args=(cur_shards, down_sample))
    p.start()
    jobs.append(p)
    
for p in jobs:
    p.join()

print("\n\nAll done.")

In [18]:
r = process_shard((True, 0, 0))

/Users/jiayou/anaconda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


In [22]:
r[0].head().T

,0,1,2,3,4
user_id,100000000.0,100000000.0,100000000.0,1.000000e+08,100000000.0
product_id,23.0,79.0,1559.0,2.002000e+03,2573.0
last,115.0,13.0,43.0,7.100000e+01,101.0
prev1,NaN,NaN,28.0,3.000000e+01,NaN
prev2,NaN,NaN,30.0,6.000000e+00,NaN
median,NaN,NaN,29.0,8.000000e+00,NaN
mean,NaN,NaN,31.5,1.466667e+01,NaN
